In [4]:
import datasets
import google.cloud.storage as gcs
import os
import json

In [5]:
import sys; sys.path.append("..")
from secret_cfg import PROJECT_ID

In [6]:
data_dir = "data"

In [7]:
if not os.path.exists("../" + data_dir):
    os.makedirs("../" + data_dir)

In [8]:
for split in ["train", "validation"]:
    dataset = datasets.load_dataset('sst2', split=split)
    dataset.to_csv(f"../{data_dir}/ds_{split}.csv")

Found cached dataset sst2 (/home/jupyter/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5)


Creating CSV from Arrow format:   0%|          | 0/68 [00:00<?, ?ba/s]

Found cached dataset sst2 (/home/jupyter/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5)


Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [9]:
client = gcs.Client(project=PROJECT_ID)
bucket = client.bucket("pytorch-training-with-vertexai")

for f in os.listdir("../" + data_dir):
    if f[-3:] == "csv":
        gcs_filename = "training_data/en" + "/" + f
        blob = bucket.blob(gcs_filename) 
        blob.upload_from_filename("../" + data_dir + "/" + f)

In [10]:
labels = dataset.data.column("label").to_pylist()

In [11]:
d = {
    0: "negative",
    1: "positive"
}
with open("../" + data_dir + "/label_map.json", "w") as f:
    json.dump(d, f)

In [12]:
gcs_filename = "model_info/en/label_map.json"
blob = bucket.blob(gcs_filename) 
blob.upload_from_filename("../" + data_dir + "/label_map.json")